<h1 style="text-align:center;">Лабораторная работа №2</h1>

<h3 style="text-align:center;">Тулупов Николай Дмитриевич</h3>
<h3 style="text-align:center;">Б01-204</h3>

<span style="color:green">

Цель: решить выбранную СЛАУ следующими методами (постараться избежать умножения матриц, использовать поэлементные записи): 
1. метод Гаусса с выбором главного элемента
2. метод LU-разложение (если применим)
3. метод Якоби
4. метод Зейделя
5. метод верхней релаксации
6. метод градиентного спуска
7. метод минимальных невязок
8. стабилизированный метод бисопряженных градиентов

*Для итерационных методов получить график убывания невязки в зависимости от итерации.</span>

In [69]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(precision=3, suppress=True, formatter={'all': lambda x: f'{x:0.3f}'})


In [70]:
a = 20 #константа - зависит от варианта задания
size = 5 #размер матриц
A = ((a-1)*np.eye(size) + np.ones(size)) #матрица А
f = np.zeros((size, 1)) 
for i in range(size): f[i] = i + 1 # столбец решений f

In [71]:
def norm3_vect(vect):
    return pow(sum(el**2 for el in vect), 0.5)


In [72]:
def norm3_matr(matr):
    size = len(matr)
    ans = 0
    for i in range(size):
        ans += sum(matr[i][j] ** 2 for j in range(size))
    return ans

In [73]:
def swap(a, b):
    return b, a
    
'''
почему не работает смена путем запаковки и распаковки кортежа
 для смены элементов массива и матрицы??????
'''

def swap_rows(A, row1, row2):
    A[[row1, row2]] = A[[row2, row1]]

def swap_columns(A, col1, col2):
    A[:, [col1, col2]] = A[:, [col2, col1]]

def find_max_el(A, iteration):
    size = A.shape[0]
    main_element = A[iteration,iteration]
    i_main = iteration
    j_main = iteration

    for i in range(size - iteration): 
        for j in range(size - iteration):
            if abs(A[i + iteration,j + iteration]) > abs(main_element):
                i_main = i + iteration
                j_main = j + iteration
                main_element = A[i_main,j_main]
    return main_element, i_main, j_main

In [74]:
def gauss(A, f):
    print("Решение СЛАУ методом Гаусса с выбором макс элемента")
    size = len(A)
    if A.shape[0] != A.shape[1]:
        print("Матрица не квадратная, решение невозможно!")
        return 
    x = np.arange(size) #массив с порядком корней (порядок будет меняться при перестановке столбцов)

    #прямой ход алгоритма - приводим матрицу A к верхнетреугольному виду
    for iter in range(len(A)):
        ''' Добавим в алгоритм проверку - если на какой-то итерации алгоритма возникла нулевая строка - ответ будет выражаться 
        через одну из переменных (её берем за константу, все остальные переменные будут выражены через нее)'''
        if (len(A) != sum(int(np.any(el)) for el in A)):
            '''Количество переменных превышает количество уравнений, решение не однозначн'''
            print("Матрица не квадратная, решение невозможно!")
            return 
        
        main_el, i_main, j_main = find_max_el(A, iter) #максимальный элемент текущей матрицы и его местонахождение
        
        if (i_main != iter): #если элемент еще не в нулевой строке
            swap_rows(A, iter, i_main) #меняем в текущей матрице нулевую строку и строку,   содержащую главный элемент            
            #f[i_main], f[iter] = swap(f[iter], f[i_main])
            temp = f[iter]
            f[iter] =  f[i_main]
            f[i_main] = temp
            #f[iter], f[i_main]  =  f[i_main], f[iter] #то же самое - в столбце решений
            
        if (j_main != iter): #если элемент еще не в нулевом столбце
            swap_columns(A, iter, j_main) #меняем в текущей матрице нулевой столбец и столбец, содержащий главный элемент
            #x[iter], x[j_main] = x[j_main], x[iter] #то же самое - в строке порядка переменных
            temp = x[iter]
            x[iter] =  x[j_main]
            f[j_main] = temp
        
        if main_el != 0:
            A[iter:, iter:][0] /= main_el #делим нулевую строку текущей матрицы на главный элемент
            f[iter] /= main_el #делим нулевую строку текущего столбца решений на главный элемент

        for i in range(size - iter - 1):   
            f[i+iter+1] -= (f[iter] * A[iter:, iter:][i+1][0])
            #вычитаем из каждого элемента нулевой в текущем столбце решений, умноженный на нулевой элемент нужной строки - получаем в начале каждой строки единицу
            A[iter:, iter:][i+1] -=  (A[iter:, iter:][0] * A[iter:, iter:][i+1][0])
            #вычитаем из каждой строки нулевую, умноженную на нулевой элемент нужной строки - получаем в начале каждой строки единицу
        
    f[-1] /= A[-1][-1]
    A[size-1][size-1] = 1

    U  = np.zeros((size, 1)) #столбец решений

    #обратный ход алгоритма
    for i in range(size-1, -1, -1):
        U[i] = f[i]
        for j in range(i + 1, size):
            U[i] -= U[j] * A[i][j]

    #перестановка переменных в изначальном порядке
    ans = np.zeros((size, 1))
    for i in range(size):
        ans[int(x[i])] = U[i]
    
    return ans

In [ ]:

A_gauss = copy.deepcopy(A) #сохраним исходную матрицу А в отдельной переменной
f_gauss = copy.deepcopy(f) #и столбец решений тоже сохраним в отдельной переменной
#это - т к иначе python расценит разные numpy матрицы как ссылающиеся на один объект

u = gauss(A_gauss, f_gauss) #столбец решений
print('x = \n', u) # ответ - столбец с переменными
print("\n")

print("Проверка \n") #перемножаем 
Res = np.array(A@u)
#print("A * U = f =\n", Res)
if norm3_vect(Res[i] - f[i]) > 1e-5: 
    #проверяем на приблизительное равенство из-за машинных ошибок
    print("Error")
else:
    print("OK")


2) LU - разложение

In [76]:
def LU_decomp(A):

    size = len(A)
    L = np.zeros((size,size))
    U = np.zeros((size,size))

    for i in range(size):
        L[i][i] = 1
        for j in range(i, size):
            Uij = (A[i][j] - sum(L[i][k] * U[k][j] for k in range(i)))
            '''if Uij == 0:
                print("Error! devision by 0")
                return'''
            U[i][j] = Uij
            
            L[j][i] = (A[j][i] - sum(L[j][k] * U[k][i] for k in range(i))) / U[i][i]

    '''Проверка'''
    if norm3_matr(L@U - A_lu) > 1e-5:
        print("Error! false")
        return 
    
    return L, U

In [77]:
def LU(A, f):
    print("Решение СЛАУ с помощью LU-разложения")
    size = len(A)
    
    #LU-разложение матрицы A
    LU_dec = LU_decomp(A)
    L = LU_dec[0]
    U = LU_dec[1]
    
    
    #обратный ход - решение СЛАУ
    #Ly = f 
    y = np.ones((size, 1))
    for i in range(size):
        y[i] = f[i] - sum(L[i][j] * y[j] for j in range(i))
    
    #Ux = y 
    x = np.ones((size, 1))
    for i in range(size-1, -1, -1):
        x[i] = (y[i] - sum(U[i][j] * x[j] for j in range(i+1, size))) / U[i][i]
    
    return x

In [ ]:
A_lu = np.copy(A) # как в пред пункте сохраним исходную матрицу А в отдельной переменной
f_lu = np.copy(f) #и столбец решений тоже сохраним в отдельной переменной

print("A=", A_lu, sep='\n')   
print("f=", f_lu, sep='\n') 

ans = LU(A_lu, f_lu)


print("x=", ans, sep='\n')

'''Проверка'''
if norm3_vect(A_lu@ans - f_lu) > 1e-5:
    print("Error! false") 
else:
    print("OK")


3) Метод верхней релаксации